In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import glob, os
import psutil
import time
from scipy.io import loadmat
from scipy.io import savemat # store results for comparison with Matlab code   

print('loading data')
spikes = loadmat('/home/mackelab/Desktop/Projects/Stitching/results/cosyne_poster/gb_net/spikes_20trials_10msBins')
spikes= spikes['spikes_out']

num_iter = 0
pi_method = 'heuristic'

print('concatenating trials')
p, T = 1000, 600
#data = spikes[0][0].T

print('number trials: ', spikes.size)
idx_n = np.sort(np.random.choice(1000, size=p, replace=False))
y = np.vstack([spikes[i][0].T[:,idx_n] for i in range(spikes.size)]).astype(np.float)
T *= spikes.size
del spikes

T,p = y.shape
idx_shuffle = np.arange(p)
np.random.shuffle(idx_shuffle)
y = y[:,idx_shuffle]


plt.figure(figsize=(20,20))
plt.imshow(y[1000:2000,:].T, aspect='auto', interpolation='None', cmap='gray')
plt.show()

In [ ]:
from ssidid.SSID_Hankel_loss import run_bad, plot_slim, print_slim, f_l2_Hankel_nl, f_l2_Hankel_comp_Q_Om
from ssidid.utility import get_subpop_stats, gen_data
from ssidid import ObservationScheme
from ssidid.icml_scripts import run_default


p,n = 1000, 10
lag_range = np.arange(20)
overlap = 1000
sso=True
mmap, data_path = False, None
idx_a, idx_b = np.arange(p), np.arange(p)
sub_pops = (np.arange((p+overlap)//2),np.arange((p-overlap)//2,p))

reps = 1
obs_pops = np.concatenate([ np.arange(len(sub_pops)) for r in range(reps) ])
obs_time = np.linspace(0,T, len(obs_pops)+1)[1:].astype(int)
obs_scheme = ObservationScheme(p=p, T=T, 
                                sub_pops=sub_pops, 
                                obs_pops=obs_pops, 
                                obs_time=obs_time)

W = obs_scheme.comp_coocurrence_weights(lag_range, sso=sso, idx_a=idx_a, idx_b=idx_b)
if overlap < p:
    for m in range(len(lag_range)):
        W[m][0,1] = 0
        W[m][1,0] = 0

print('computing time-lagged covariances')
Qs, Om = f_l2_Hankel_comp_Q_Om(n=n,y=y,lag_range=lag_range,obs_scheme=obs_scheme,
                      idx_a=idx_a,idx_b=idx_b,W=W,sso=sso,
                      mmap=mmap,data_path=data_path,ts=None,ms=None)    
if overlap < p:
    for m in range(len(lag_range)):    
        Om[m][np.ix_(obs_scheme.idx_grp[0], obs_scheme.idx_grp[1])] = False
        Om[m][np.ix_(obs_scheme.idx_grp[1], obs_scheme.idx_grp[0])] = False


rnd_seed = np.random.get_state()
#np.random.seed(rnd_seed)
pars_est, traces, ts= run_default(
            alphas    = (0.1, 0.01), 
            b1s       = (0.9, 0.95), 
            a_decays  = (0.95, 0.98), 
            batch_sizes = (1, 10), 
            max_zip_sizes =  (100,100), 
            max_iters = (100, 100 ),
            parametrizations = ('nl', 'nl'),
            pars_est='default', pars_true=None, n=n, 
            y=y, sso=sso, obs_scheme=obs_scheme, lag_range=lag_range, 
            idx_a=idx_a, idx_b=idx_b,Qs=Qs,Om=Om, W=W,
            traces=[[], [], []], ts = [])    

In [ ]:
rnd_seed = np.random.get_state()
#np.random.seed(rnd_seed)
pars_est, traces, ts= run_default(
            alphas    = (0.1, 0.01), 
            b1s       = (0.9, 0.95), 
            a_decays  = (0.95, 0.98), 
            batch_sizes = (1, 10), 
            max_zip_sizes =  (100,100), 
            max_iters = (100, 100 ),
            parametrizations = ('nl', 'nl'),
            pars_est='default', pars_true=None, n=n, 
            y=y, sso=sso, obs_scheme=obs_scheme, lag_range=lag_range, 
            idx_a=idx_a, idx_b=idx_b,Qs=Qs,Om=Om, W=W,
            traces=[[], [], []], ts = [])    